> ### Outline

- 선형 분해(Matrix Factorization)
    - 선형 분해의 4가지 방식
- 딥러닝 기반 모델
    - VAE 기반 모델
    - GNN 기반 모델
    - Transformer 기반 모델
- 최근 딥러닝 추천시스템 트렌드
    - 선형 모델 vs 딥러닝 모델
    - 서로 차이점
    - 왜 같이 사용되는가?

> ### 0. 추천 시스템 기초 정리

- 내용 기반 필터링  
**콘텐츠의 유사도**가 바탕 -> 사용자가 선택한 아이템과 유사한 아이템을 추천  

- 협업 필터링  
**사용자 간의 유사도**가 바탕  
    - 사용자 기반: 특정 사용자와 유사한 취향의 사용자가 선호한 아이템  
    - 아이템 기반: 사용자가 선호한 아이템과 유사한 다른 아이템 추천  
    (위의 내용 기반 필터링은 아이템들 자체의 유사성을 측정하고, 이것은 아이템에 대한 사용자들의 평가/선호 패턴의 유사성을 측정하는 것이 차이점)

- 하이브리드 필터링  
2가지 이상의 추천시스템 알고리즘을 조합

> ### 0-1. 협업 필터링의 구현 방식

1. 기억 기반 협업 필터링
    - 사용자 기반 작동 방식  
        a. 모든 사용자 간 유사도 계산  
        b. 유사도가 높은 상위 k명의 이웃 선정  
        c. 이웃이 선호했지만 내가 아직 보지 않은 아이템 추천

    - 아이템 기반 작동 방식  
        a. 아이템 쌍의 유사도를 계산  
        b. 내가 선호한 아이템과 유사한 아이템 추천
    
    - 문제점  
        - 희소 행렬 문제: 사용자들이 일부 아이템에만 평점을 주기 때문에, 행렬의 대부분이 비어 있게 됨  
        - 확장성 문제: 사용자와 아이템 수가 증가할수록 계산량이 급격히 늘어남  
        - 콜드 스타트 문제: 새로운 사용자/새로운 아이템에는 상호작용 내역이 없어서 추천이 어려움

2. 모델 기반 협업 필터링  
-> 기억 기반 협업필터링의 한계를 극복하기 위해 등장  
    - 모델이 잠재 요인을 학습하고, 비어있는 평점을 예측  
    - 유사도 대신 내적을 계산

> ## 1. 모델 기반 협업 필터링

> ### 1-1. 잠재 요인  

- 잠재 요인  
![잠재요인](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdna%2FlsYs7%2FbtqOcRPI3UJ%2FAAAAAAAAAAAAAAAAAAAAACiZp9dgiLQpyYsIWyRATlwSJZ75eZ4nmHvtl5iVZFlg%2Fimg.png%3Fcredential%3DyqXZFxpELC7KVnFOS48ylbz2pIh7yKj8%26expires%3D1764514799%26allow_ip%3D%26allow_referer%3D%26signature%3D6dlSpJZMHIeMN8J7I8djHwQ1fN4%253D)  
(Latent Factor Space 예시 사진과 다르게, x축과 y축이 무엇을 의미하는지 알 수 없다. 또한 차원은 여러개 일 수 있다.)

- 잠재 요인이란, 사용자-아이템 상호작용 행렬 속에서 모델이 학습한, 사용자의 취향이나 아이템의 특성을 설명하는 **숨겨진 차원**  
-> 사람이 명시적으로 이름 붙일 수 없음. 단, 간접적으로 파악할 수는 있음

- Latent Matrix factorization
![factors](https://miro.medium.com/v2/resize:fit:1276/1*pNXo4Zj_QOZLS0a2mu5OSg.jpeg)
    - 입력 행렬: $R$ (노래/유저)  
    사용자가 모든 노래를 듣거나 평점 주지 않았을 것이기 때문에, 이 R은 희소 행렬일 가능성이 높다  
    - 잠재 요인 행렬로 분해: $X^T$ 와 $Y$
        a. $X^T$ 사용자의 잠재 요인 행렬: users(행) x factors(열)  
        b. $Y$ 노래(아이템)의 잠재 요인 행렬: factors(행) x songs(열)
    - 입력 행렬의 빈칸은 두 잠재 요인 벡터의 내적으로 예측된다!  
    (* 실제로는 예측 평점이 정해진 척도(예: 1~5) 안에 들도록 복잡한 학습 과정과 편향 처리가 추가된다.)

> ### 1-2. 행렬 분해(MF)  

가설: R의 평점($r_{u,i}$)은, 사용자 u의 잠재 요인 벡터 $p_u$와 아이템 i의 잠재 요인 벡터 $q_i$의 내적 결과이다.  
목표: 관측된 평점을 가장 잘 재현하는 가장 적절한 잠재 요인 개수 $k$를 찾고,  
$R \approx P \times Q^T$가 되도록 $P$와 $Q^T$를 학습하는 것

- MF 수행 방식  
    - SVD(수학적 분해 기법): 행렬 대수학 기반, 희소 행렬에는 적용 불가(결측치 처리 필요)  
    - SGD(최적화 알고리즘): 경사 하강법 기반, 관측된 평점에 대해서만 학습  
    - NMF(제약 조건 기반 모델): 분해된 행렬에 '비음수' 제약 조건 추가 -> 잠재 요인의 해석력 높임  
    - PMF(확률적 모델): 평점 예측에 확률 분포 사용 -> 불확실성도 모델링

- SVD: 어떤 직사각형 행렬이든 세가지 행렬로 분해  
$$A=UΣV^T$$
A: 원래의 m*n 행렬  
U: m*m 직교 행렬(사용자 정보)  
V^T: n*n 직교 행렬(아이템 정보)  
Σ: m*n 대각 행렬(**특이값**)  
> - 특잇값 행렬의 역할과 중요성  
    - 대각 행렬: 대각선을 제외한 원소가 0, 대각선 순으로 A의 특잇값이 내림차순으로 배열되어 있음  
    - 특잇값: 각각의 특잇값은 해당 차원(잠재 요인)이 원본 행렬 A에서 얼마나 많은 정보량(중요도)를 설명하는지 나타냄  
    - 추천시스템에서의 중요성: 특잇값 상위 k개와 그에 대응하는 벡터만 사용한다면, 원래 행렬의 핵심 패턴을 효율적으로 압축할 수 있음  

- 여기서 Full SVD는 모든 특잇값과 벡터를 사용하고, 밀집 행렬에 적합하다.  
- Truncated SVD는 상위 k개의 큰 특잇값과 벡터를 사용하고, 희소 행렬 압축과 노이즈 제거에 유리하다.  

    - 단, SVD 자체는 결측치 처리를 요구하므로, 원본 행렬 A에서 결측치를 평균 평점 등으로 처리해야 한다.

- SGD  
SVD를 실제 서비스 환경에 적용하기에는 희소 행렬 문제가 있고, 계산이 복잡하므로, 이를 보완하기 위해 SGD 방식 고안  
    - SGD의 목표: 목적 함수  
    SGD는 관측된 평점과 예측 평점 사이의 오차를 최소화하는 것을 목적으로 함.  
    -> 오차의 제곱합을 최소화 + 과적합 방지를 위한 **정규화 항**을 추가함

    - 무작위 작은 값으로 초기화한 사용자 행렬 & 아이템 행렬  
        - epoch만큼 모든 관측된 평점을 순회  
        - 예측 평점을 예측하고, 오차 계산  
        - 경사하강법 기반으로, 오차를 줄이는 방향으로 업데이트

- NMF: 분해된 두 행렬릐 모든 원소가 0 이상이어야 한다는 조건을 추가  
    - 더 직관적이고 현실과 가까운 해석을 위함

    - 초기화 할 때 비음수인 무작위 값으로 초기화  
        - 경사하강법 대신, '곱셈 업데이트 규칙'과 같은 특수 알고리즘을 사용해 업데이트

- PMF: 확률적 모델로 접근 -> "누가 무엇을 좋아할지"에 대한 예측이 목표  
확률적 모델이기에 예측 평점과 불확실성을 포함  

---

> ## 2. 최근 추천시스템의 방향성  

> ### 2-1. VAE 기반 추천시스템

VAE는 인코더가 단일 벡터 z로 추측하는 방식(AE)이 아닌, 잠재변수 z 분포로 추측하는 방식  

- MF의 한계  
    - 선형성의 한계: 평점 예측이 단순한 잠재 벡터의 내적으로 이루어짐  
    -> 딥러닝은 비선형 패턴을 학습하는 데 강함  
    - 불확실성 무시: 예측값을 하나의 벡터로만 표현함  
    -> VAE는 확률 분포로 불확실성 또한 반영  

- 불확실성 무시 문제 해결  
    - AE 접근 방식: 벡터 하나로 추정(데이터가 부족하면 추측이 불안정하고 노이즈에 취약)  
    - VAE 접근 방식: 분포를 추정 -> 데이터 부족으로 인한 오차 범위까지 모델링에 포함 -> 일반화 안정성 높아짐

- VAE 기반 CF의 작동 구조  
    - 입력: 사용자-아이템의 상호작용 벡터  
    - 인코더: 사용자 이력을 잠재변수 z로 맵핑  
    - 잠재 벡터(z): 샘플링된 취향 벡터  
    - 디코더: z로부터 아이템 분포를 복원  

- **목적 함수**  
두 가지 목표를 동시에 최적화  
    1. 재구성 오차 최소화:  
    디코더가 잠재 벡터 z를 사용하여 원래 입력 x를 최대한 비슷하게 복원하도록 함(AE와 동일)  
    2. KL Divergence:  
    인코더에서 출력되는 잠재 분포가, 사전 분포(표준 정규분포)와 유사하도록 함  
    -> 모델이 특정 데이터에 과적합되는 것을 방지, 잠재 공간을 부드럽게 연결(유저 벡터들이 표본 분포 주변에 연결된 상태로 있도록 만듦)  
    -> 데이터가 부족한 유저 벡터에 있어서도 정규 분포를 기반으로 보간이 가능  

> 부족한 유저(희소한 데이터)라도 주변의 풍부한 데이터(정보가 많은 유저)로부터 정보를 자연스럽게 보간받아 안정적인 예측을 할 수 있게 됨  
-> VAE-CF가 희소 데이터에 강한 이유!

- VAE-CF의 주요 장점  
    - 불확실성을 모델링에 반영  
    - 희소한 상호작용 문제에 강함  
    - 비선형성 학습(신경망 구조이기 때문에)

> ### 2-2. GNN 기반 사용자 선호 예측

- GNN  
![GNN비교](https://cdn.prod.website-files.com/62528d398a42420e66390ef9/65d2fa1aef05ae76c63f563a_image2.png)

- 그래프 자료구조  
![그래프](https://images.velog.io/images/gimtommang11/post/b632e2fd-4a41-4e96-b7c8-56399615308c/graph.png)  

**노드(개체)와 엣지(개체 간 관계)로 구성된 자료구조**

- 그래프 자료구조를 사용하는 이유  
    1. 관계, 상호작용과 같은 추상적인 개념을 다루기에 적합  
    2. 비 유클리드 공간을 표현하고 학습 가능(SNS의 관계망과 같은, 보다 현실에 가까운 느낌)  

- GNN의 핵심 아이디어  
    **나와 이웃한 노드(사용자, 아이템)의 정보를 나에게 전달/요약(=메세지 패싱)**  

> GNN의 작동 구조

1. 그래프 구조 생성  
노드와 엣지를 정의 -> 각각의 피쳐를 담아서 표현  
예) 사용자와 아이템 정보를 **노드**를 생성 -> **엣지**는 사용자와 아이템 사이의 상호작용을 의미하도록 연결  

2. 메세지 패싱  
그래프에서 노드 간 정보를 전달하고 상호작용하는 단계  
하나의 GNN 레이어에서 메시지 생성 - 집계 - 갱신의 세 단계로 이루어짐.  
GNN 모델이 k개의 레이어로 구성되어 있다면, 위 세 단계가 k번씩 이루어짐

    0. 타겟 노드 v를(정보를 갱신하려는 노드) 선정  
    이때, 모든 노드가 돌아가면서 타겟 노드가 됨  

    a. 메세지 생성  
    이웃에 전달할 메시지 생성  
    메시지 함수를 통해, 이웃에게 보내는 자신 노드의 정보(임베딩 형태)를 생성

    b. 집계  
    v(타겟 노드)의 이웃 노드들의 임베딩을 요약본으로 만듦
    여러 이웃 노드에서 받은 메시지(임베딩)를 하나의 벡터로 통합

    c. 갱신  
    v의 원래 임베딩과 집계 결과를 합쳐서 새 정보로 갱신  
    자신의 이전 상태 + 집계된 이웃 정보 -> 새로운 임베딩을 계산

3. 최종 출력  


> GNN 아키텍처 종류  

GCN: 그래프 위에서 Convolution 연산 수행  
GRN: RNN + GNN  
GAT: 이웃 노드 간 중요도를 학습해 가중 평균 (타겟 노드의 이웃 노드들 중 중요한 노드에 가중치를 줌)  
GraphSAGE: 대규모 그래프에 대해 샘플링 기반으로 학습

- LightGCN: 추천시스템에 특화된 가벼운 GCN  
GNN의 복잡한 연산이 오히려 상호작용 정보를 왜곡할 수 있다는 아이디어에서 탄생  

    1. 특징 변환(Feature Transformation) 제거  
    레이어마다 복잡했던 학습 파라미터($W^l$) 삭제  
    2. 비선형 활성화 함수 제거  
    그 대신 이웃 정보를 합치는 방법 사용(주로 정규화된 합 or 평균으로 간단하게)

    -> 오직 **정규화**된 **이웃 임베딩의 합**이라는 추천시스템의 본질에 가까운 모델!

- 사용 예시  
핀터레스트와 같은 대규모 서비스에서의 추천 모델  
소셜 네트워크  
화학 및 생명과학  
트래픽(도로 구조는 비유클리드에 가까움)

- GNN의 장단점  
    - 장점  
    고차원의 협업 시그널 포착  
    관계(상호작용) 자체를 학습하기 때문에, 노드 자체의 특성이 또렷하지 않아도 관계망 정보를 전파 가능  
    콜드 스타트 문제 완화에 효과적  

    - 단점  
     - 과평탄화(Over-smoothing) 문제  
     메시지 전달이 반복되어 GNN의 레이어가 많아질수록, 그래프 안의 모든 노드 간의 임베딩이 비슷해지는 문제  
     - 과압축 문제  
     GNN의 층이 깊어질수록 집계하는 이웃의 범위가 커짐 -> 고정 크기의 벡터에 압축하나 보니 정보가 손실됨  
     - 계산 복잡도, 컴퓨팅 자원 요구  
     - 해석의 어려움  
     GNN은 상호작용을 집합으로는 보지만, 순서를 반영하지는 않음. (실제 세계에서는 사용자의 의도가 순서에 반영되어 있을 수도 있음)

> ### 2-3. 트랜스포머 기반 추천 시스템  

- 기존 추천 시스템의 순차적 한계  
    - CF의 순차적 한계: 순서 무시, 시간대에 대한 문맥 부족, 예측의 한계(다음 행동을 예측하는 것이 아닌, 일반적인 선호도를 찾는 것이 목적)  
    - GNN의 순차적 한계: 여러 노드를 거치면서 정보가 희석될 수 있음, 장기 의존성 문제  

    -> 행동의 순서, 문맥, 동적인 의도를 파악하는 데 약하다.  
    -> '사용자의 행동 시퀀스'를 이해하고 다음 행동을 예측하는 트랜스포머 모델이 대안으로 등장  

- 사용자의 행동 순서(문맥)가 중요한 이유: 사용자 의도의 파악  

1. 문맥에 따라 달라지는 **동일 아이템의 의미**  

문맥 A (피크닉/야외 활동 준비)  

- 행동 순서: [클릭: 돗자리] → [클릭: 도시락] → [클릭: 텀블러]
- Transformer의 해석: "텀블러"가 돗자리, 도시락와 강하게 연관(Attention)  
    이때 "텀블러"는 **야외용 물통/보온병**으로 해석됨  
- 다음 추천: 휴대용 블루투스 스피커, 접이식 캠핑 의자, 보냉백

문맥 B (사무실 환경 구성)

- 행동 순서: [클릭: 노트북 거치대] → [클릭: 무소음 마우스] → [클릭: 텀블러]  
- Transformer의 해석: "텀블러"가 노트북 거치대, 무소음 마우스와 강하게 연관(어텐션)  
    이때 "텀블러"는 '책상 위 개인용 음료 용기' 또는 **업무 중 집중을 돕는 아이템**으로 해석됨  
- 다음 추천: 문구 정리 트레이, 의자용 쿠션, 드립백 커피

2. 시간 순서를 뛰어넘는 핵심 의도 포착  
멀리 떨어진 과거 행동이 다음 행동의 키가 될 수 있음  

- 행동 순서: [구매: 선글라스] → [클릭: A4용지] (생활) → [구매: 달걀] (생활)... → [검색: 스마트폰 방수팩]  
- Transformer의 해석: "방수팩"이 쿼리가 되고, 이 쿼리는 A4용지, 달걀이 아닌 "선글라스"(키)와 가장 강한 어텐션 관계  
- 다음 추천: 모자, 샌들

3. 셀프 어텐션 메커니즘  
시퀀스 내의 모든 아이템 간의 관계를 거리에 상관없이 동시 계산  
    -> "더 오래된 클릭과 관계가 있을 수 있다"는 직관을 모델에 반영

- 쿼리(Q): 관련 정보를 요청(지금 보고 있는 상품과 관련된 정보?)  
- 키(K): 정보의 종류를 나타내는 태그(시퀀스 내 다른 상품들을 나타내는 키)  
- 밸류(V): 키와 연결된 내용물(실제 상품)

- 연산 과정  
    - 어텐션 점수 계산: 주로 Q와 K의 내적으로 계산  
    - 가중합 계산: 어텐션(관련성) 점수를 Softmax 함수를 거쳐 (합치면 1이 되는)가중치로 변환  
    - 가중 평균 계산: 가중치를 밸류와 곱함  
    -> 아이템 간 관련성에 따라 정보 반영률이 차등 적용!

    즉, 현재 클릭한 "방수팩"과 관련성이 높은 '선글라스'의 정보(V)는 강하게 반영되고, 관련성이 낮은 "A4용지"의 정보는 약하게 반영된, **새로운 '문맥 벡터'**가 생성됨

> 대표적인 트랜스포머 기반 추천시스템

- SASRec  
    - 학습용 행동 순서(사용자가 상호작용한 아이템의 순서)를 모델에 입력  
    - 임베딩 레이어: 아이템을 벡터 형태로 변환  
    - 셀프 어텐션 레이어: 모델이 시퀀스 내의 아이템 간 관계를 학습  
    - 피드포워드 인공 신경망(FNN): 비선형 변환을 한 번 거쳐 예측 능력을 높임  
    - 예측 레이어: 모델의 학습을 바탕으로, 시퀀스의 다음 단계 아이템을 예측  

    단방향 어텐션 방식으로, (GPT와 유사) 과거의 아이템만을 참고하는 것이 특징  
    - 문제점: 계산량이 많음, 단방향성

- BERT4Rec  
    - 입력: 사용자의 행동 시퀀스 중 일부를 [MASK] 토큰으로 마스킹  
    - 임베딩 레이어: 아이템 임베딩(아이템을 벡터화), 위치 임베딩(아이템의 순서 정보를 담은 벡터)  
    -> 두 벡터를 덧셈하여 트랜스포머의 입력으로 사용  
    - 트랜스포머 블록: '양방향' 셀프 어텐션(BERT 방식)  
    마스킹된 토큰을 맞추기 위해, 그 위치의 과거와 미래 문맥을 모두 참조  
    - 예측: 트랜스포머 층을 통과한 [MASK] 토큰의 **벡터**가 예측 레이어를 거쳐서, 원래 아이템이 무엇이었는지를 예측  

    양방향 어텐션 방식

    마스킹을 하는 이유?  
    - 양방향으로 학습할 때 직접 서로의 정답을 참고하는 누수 문제를 방지하기 위해서

    특징  
    - 아이템의 양방향 문맥(과거와 미래)을 모두 고려  
    - '시퀀스 전체'의 선호도 파악에 더 초점이 맞춰져 있음

- 사용자와 아이템의 상호작용 수가 많으면, BERT4Rec이 넓은 문맥 파악에 좋다.  
- 상호작용 수가 적다면, 기록이 별로 없는데 굳이 복잡하게 앞뒤를 다 따지는 것보다, SASRec으로 비교적 단순하게 접근하는 것이 더 효율적.

---

> ## 3. 현업 트렌드

> ### 3-1. LLM 활용  

- 관점의 전환 -> 추천을 랭킹 문제가 아닌 **언어** 문제로  
텍스트, 이미지, 오디오 등 멀티모달 형태로 전달하거나, 사용자의 상황에 맞는 이유를 덧붙인 서술형 추천도 가능
    - 대화형/동적 추천  
    - Zero-Shot 추천: 학습 데이터에 없는 새로운 아이템도 텍스트 설명을 통해 그럴싸한 추천 가능(콜드 스타트 문제 해결책)

- 활용: P5/TALLRec  
    - P5: 모든 추천 과제를 LLM이 풀 수 있도록 거대한 텍스트 프롬포트로 통일시킨 모델  
    - TALLRec: P5에는 비용 문제가 있으므로, 기존 범용 LLM을 추천 작업에 맞게 튜닝하는 프레임워크

> ### 3-2. 책임감 있는 AI  

- 인기 편향  
추천 시스템이 소수의 인기 아이템을 과도하게 노출하고, 비인기 아이템에는 관심이 쏠리지 않는 현상

- 설명 가능성(XAI)  
추천의 이유를 설명할 수 있는가?에 대한 문제  
딥러닝 모델은 **블랙박스 문제**가 있음  

-> 최고의 모델은 가장 정확한 모델이 아니라, 편향과 공정성을 고려하고, 추천 근거를 설명할 수 있는 책임감 있는 모델  

> ### 3-3. 딥러닝 도입의 비용 문제

- 컴퓨팅 자원과 비용 문제로 인해, 선형 기반 추천시스템 또한 현업에서 병행되는 중  

1. 후보군 생성 - 선형 모델
선형 모델의 **빠른 계산속도**와 **적은 학습비용**를 바탕으로, 전체 아이템군에서 사용자가 좋아할 만한 아이템을 추출  
-> 2단계에 전달  

2. 순위 매기기 - 딥러닝 모델  
전달받은 후보군을 정교하게 순위로 매김  
높은 **정확도**, **복잡한 패턴** 학습에 유리함